# Question 1

In [3]:
using DataFrames
df = readtable("uy_data.csv",header=false)

,x1,x2
1,-0.9188,-1.0773
2,-0.6159,-0.9419
3,0.1959,-0.0915
4,-0.0316,0.0245
5,-1.3455,-1.3239
6,-0.0513,-0.3536
7,0.1471,0.1032
8,-1.1615,-1.0702
9,0.5478,0.2722
10,0.6096,0.9127


In [22]:
y_vals = []
for val in df[:,1]
    append!(y_vals,val)
end
y_vals

100-element Array{Any,1}:
 -0.9188
 -0.6159
  0.1959
 -0.0316
 -1.3455
 -0.0513
  0.1471
 -1.1615
  0.5478
  0.6096
 -0.9706
  0.3018
  0.2039
  ⋮     
  0.205 
 -0.8645
  0.1419
  0.2251
 -0.4831
 -0.1577
 -0.7281
  0.1397
  0.0851
 -0.2105
 -0.011 
 -0.2845

In [23]:
k  = 5
using JuMP, Gurobi


m = Model()
@variable(m,y[1:length(y_vals)])
@variable(m,y_pred[1:length(y_vals)])
@variable(m, b[1:k])

@constraint(m, )[]
solve(m)

INFO: Recompiling stale cache file /home/max/.julia/lib/v0.5/Clp.ji for module Clp.


:Optimal